## LMFlow medical model test

- LMFlow : https://github.com/OptimalScale/LMFlow

In [ ]:
# !cd LMFlow; pip install -e .

In [ ]:
llama_13b_hf_path = "/home/ec2-user/SageMaker/efs/aiml/llama/models/13B-hf"

In [ ]:
apply_delta_script = "./LMFlow/examples/merge_lora.py"

In [ ]:
delta_path = "./pretrained-models/lmflow-medical/medical_ckpt/llama_13b_lora"

In [ ]:
target_path = "./pretrained-models/lmflow-medical/13b"

In [ ]:
# This apply_delta script is different with StableVicuna merge script
# Merge delta with LLaMA model
# !python {apply_delta_script} --base {llama_13b_hf_path} --target {target_path} --delta {delta_path}

# Merge LoRA adapter to Base model
# LLaMA need to be a HF format first
!python {apply_delta_script} \
    --model_name_or_path {llama_13b_hf_path} \
    --lora_model_path {delta_path} \
    --output_model_path {target_path}

In [ ]:
import os
os.listdir(target_path)

In [ ]:
model_location = target_path

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(model_location)
model = AutoModelForCausalLM.from_pretrained(model_location, low_cpu_mem_usage=True, load_in_8bit=True, device_map="auto")


In [ ]:
prompt = """\
### Human: I have a patient with bronchial inflammation, and I need you to write a report on the patient's condition that sounds like it was written by a doctor.
### Assistant:
"""

# prompt = "I have a patient with bronchial inflammation, and I need you to write a report on the patient's condition that sounds like it was written by a doctor."


In [ ]:
print(prompt)
inputs = tokenizer(prompt, return_tensors='pt').to('cuda')

In [ ]:
%%time
tokens = model.generate(
 **inputs,
 max_new_tokens=256,
 do_sample=True,
 temperature=0.3,
 top_p=0.3,
)


In [ ]:
result = tokenizer.decode(tokens[0], skip_special_tokens=True)
print(result)

### Medical 분야 결과

- 전혀 좋지 않은데, 프롬프트를 만들어서 넣어야 하는 듯 하는걸까? 어떤 prompt를 넣어야 원하는 게 나오는지 확인이 필요함
- 학습 데이터에 있던 예시들을 참고해서 만들어서 넣어야 하는 듯 ?
